# 🔍 Deep Dive: Symmetry Analysis

This notebook explores symmetry and palindromic structures in musical compositions.

**Topics covered:**
- Time palindromes vs. pitch palindromes
- Pairwise symmetry mapping
- Analyzing Bach's Crab Canon
- Creating custom palindromic canons
- Quantifying symmetry
- Visualizing palindromic structures

In [ ]:
# Imports
import sys
sys.path.insert(0, '..')

from cancrizans import (
    is_time_palindrome,
    pairwise_symmetry_map,
    load_bach_crab_canon,
    assemble_crab_from_theme,
    retrograde,
    mirror_canon
)
from music21 import note, stream, chord
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import FancyArrowPatch, Circle

print("✓ Imports successful!")

## 1. What is a Time Palindrome?

A **time palindrome** in music is when notes appear in symmetric temporal positions.

In [ ]:
# Create a simple time palindrome
theme = stream.Part()
theme.append(note.Note('C4', quarterLength=1.0))
theme.append(note.Note('D4', quarterLength=1.0))
theme.append(note.Note('E4', quarterLength=1.0))

# Create palindrome by mirroring
palindrome = mirror_canon(theme)

print("Theme:", [n.nameWithOctave for n in theme.notes])
print("\nPalindrome parts:")
for i, part in enumerate(palindrome.parts):
    print(f"  Part {i+1}:", [n.nameWithOctave for n in part.notes])

# Verify it's a palindrome
is_palin = is_time_palindrome(palindrome)
print(f"\nIs time palindrome? {is_palin}")

## 2. Pairwise Symmetry Mapping

This shows which notes correspond to each other in the palindrome.

In [ ]:
# Get symmetry map
symmetry_map = pairwise_symmetry_map(palindrome)

print("Pairwise Symmetry Map:")
print(f"Total note pairs: {len(symmetry_map)}\n")

for i, (note1, note2) in enumerate(symmetry_map[:10]):  # Show first 10
    t1, p1 = note1['time'], note1['pitch']
    t2, p2 = note2['time'], note2['pitch']
    print(f"Pair {i+1}:")
    print(f"  Note 1: t={t1:.2f}, pitch={p1}")
    print(f"  Note 2: t={t2:.2f}, pitch={p2}")
    print(f"  Δtime = {abs(t2-t1):.2f}")
    
if len(symmetry_map) > 10:
    print(f"... and {len(symmetry_map) - 10} more pairs")

## 3. Bach's Crab Canon Analysis

Let's analyze the authentic Bach canon.

In [ ]:
# Load Bach's canon
bach = load_bach_crab_canon()

print("Bach's Crab Canon (BWV 1079):")
print(f"Parts: {len(bach.parts)}")
print(f"Duration: {bach.duration.quarterLength} quarter notes")
print(f"Total notes: {sum(len(part.notes) for part in bach.parts)}")

# Verify palindrome
is_palin = is_time_palindrome(bach)
print(f"\nIs perfect time palindrome? {is_palin}")

# Get symmetry map
bach_symmetry = pairwise_symmetry_map(bach)
print(f"Symmetry pairs: {len(bach_symmetry)}")

## 4. Symmetry Statistics

Quantify the symmetry properties.

In [ ]:
def analyze_symmetry_stats(symmetry_map):
    """Calculate statistics about symmetry pairs."""
    time_diffs = []
    pitch_diffs = []
    matching_pitches = 0
    
    for note1, note2 in symmetry_map:
        t_diff = abs(note2['time'] - note1['time'])
        p_diff = abs(note2['pitch'] - note1['pitch'])
        
        time_diffs.append(t_diff)
        pitch_diffs.append(p_diff)
        
        if note1['pitch'] == note2['pitch']:
            matching_pitches += 1
    
    return {
        'time_separation_mean': np.mean(time_diffs),
        'time_separation_std': np.std(time_diffs),
        'time_separation_max': np.max(time_diffs),
        'pitch_diff_mean': np.mean(pitch_diffs),
        'pitch_diff_std': np.std(pitch_diffs),
        'matching_pitch_ratio': matching_pitches / len(symmetry_map)
    }

# Analyze Bach's canon
stats = analyze_symmetry_stats(bach_symmetry)

print("Symmetry Statistics (Bach's Crab Canon):")
print(f"\nTemporal Symmetry:")
print(f"  Mean separation: {stats['time_separation_mean']:.2f} quarters")
print(f"  Std deviation: {stats['time_separation_std']:.2f} quarters")
print(f"  Max separation: {stats['time_separation_max']:.2f} quarters")
print(f"\nPitch Symmetry:")
print(f"  Mean pitch diff: {stats['pitch_diff_mean']:.2f} semitones")
print(f"  Std deviation: {stats['pitch_diff_std']:.2f} semitones")
print(f"  Matching pitches: {stats['matching_pitch_ratio']*100:.1f}%")

## 5. Visualizing Symmetry

Create a symmetry diagram showing note pairs.

In [ ]:
def plot_symmetry_diagram(score, max_pairs=30):
    """Plot symmetry pairs with connecting lines."""
    symmetry_map = pairwise_symmetry_map(score)
    
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # Plot notes
    for part_idx, part in enumerate(score.parts):
        times = [n.offset for n in part.notes]
        pitches = [n.pitch.midi for n in part.notes]
        colors = ['#3498db', '#e74c3c'][part_idx % 2]
        ax.scatter(times, pitches, s=100, c=colors, alpha=0.6, 
                  label=f'Part {part_idx + 1}', zorder=3)
    
    # Draw symmetry connections (sample for clarity)
    step = max(1, len(symmetry_map) // max_pairs)
    for i, (note1, note2) in enumerate(symmetry_map[::step]):
        t1, p1 = note1['time'], note1['pitch']
        t2, p2 = note2['time'], note2['pitch']
        
        # Draw connecting arc
        ax.plot([t1, t2], [p1, p2], 'g-', alpha=0.2, linewidth=1, zorder=1)
        
    # Find midpoint time
    max_time = max(n['time'] for pair in symmetry_map for n in pair)
    midpoint = max_time / 2
    
    # Draw midpoint line
    ax.axvline(midpoint, color='red', linestyle='--', linewidth=2, 
              alpha=0.5, label='Symmetry axis', zorder=2)
    
    ax.set_xlabel('Time (quarter notes)', fontsize=12)
    ax.set_ylabel('Pitch (MIDI number)', fontsize=12)
    ax.set_title('Palindromic Symmetry Visualization', fontsize=14, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    return fig

# Plot simple palindrome
fig = plot_symmetry_diagram(palindrome)
plt.savefig('../examples/symmetry_simple.png', dpi=150, bbox_inches='tight')
print("✓ Saved: symmetry_simple.png")
plt.show()

## 6. Bach's Canon Symmetry

Visualize the full Bach canon symmetry.

In [ ]:
# Plot Bach's canon (sample pairs for clarity)
fig = plot_symmetry_diagram(bach, max_pairs=50)
plt.savefig('../examples/bach_symmetry_analysis.png', dpi=150, bbox_inches='tight')
print("✓ Saved: bach_symmetry_analysis.png")
plt.show()

## 7. Creating Asymmetric Canons

What happens when we break the symmetry?

In [ ]:
# Create an asymmetric "canon"
part1 = stream.Part()
for p in ['C4', 'D4', 'E4', 'F4']:
    part1.append(note.Note(p, quarterLength=1.0))

part2 = stream.Part()
# Different notes - NOT a retrograde!
for p in ['G4', 'A4', 'B4', 'C5']:
    part2.append(note.Note(p, quarterLength=1.0))

asymmetric = stream.Score()
asymmetric.append(part1)
asymmetric.append(part2)

# Test for palindrome
is_palin = is_time_palindrome(asymmetric)
print(f"Is asymmetric 'canon' a palindrome? {is_palin}")
print("\nExpected: False (pitches don't match in retrograde positions)")

## 8. Temporal vs. Pitch Symmetry

Compare different types of symmetry.

In [ ]:
def check_temporal_symmetry(score):
    """Check if note onsets are symmetric."""
    all_times = []
    for part in score.parts:
        all_times.extend([n.offset for n in part.notes])
    
    all_times_sorted = sorted(all_times)
    reversed_times = all_times_sorted[::-1]
    
    max_time = max(all_times)
    mirrored_times = [max_time - t for t in reversed_times]
    
    return all(abs(t1 - t2) < 0.001 for t1, t2 in zip(all_times_sorted, mirrored_times))

def check_pitch_symmetry(score):
    """Check if pitches are symmetric."""
    if len(score.parts) != 2:
        return False
    
    pitches1 = [n.pitch.midi for n in score.parts[0].notes]
    pitches2 = [n.pitch.midi for n in score.parts[1].notes]
    
    return pitches1 == pitches2[::-1]

# Test on Bach's canon
temp_sym = check_temporal_symmetry(bach)
pitch_sym = check_pitch_symmetry(bach)

print("Bach's Crab Canon:")
print(f"  Temporal symmetry: {temp_sym}")
print(f"  Pitch symmetry: {pitch_sym}")
print(f"  Perfect palindrome: {is_time_palindrome(bach)}")

# Test on asymmetric
print("\nAsymmetric 'Canon':")
print(f"  Temporal symmetry: {check_temporal_symmetry(asymmetric)}")
print(f"  Pitch symmetry: {check_pitch_symmetry(asymmetric)}")
print(f"  Perfect palindrome: {is_time_palindrome(asymmetric)}")

## 9. Quantifying Palindrome Quality

Create a metric for "how palindromic" a piece is.

In [ ]:
def palindrome_score(score):
    """Score from 0-1 indicating palindrome quality."""
    if len(score.parts) < 2:
        return 0.0
    
    symmetry_map = pairwise_symmetry_map(score)
    if not symmetry_map:
        return 0.0
    
    # Count matching pitches
    matches = sum(1 for n1, n2 in symmetry_map 
                  if n1['pitch'] == n2['pitch'])
    
    return matches / len(symmetry_map)

# Score various canons
print("Palindrome Quality Scores (0-1):")
print(f"Bach's Crab Canon: {palindrome_score(bach):.3f}")
print(f"Simple palindrome: {palindrome_score(palindrome):.3f}")
print(f"Asymmetric 'canon': {palindrome_score(asymmetric):.3f}")

print("\n✓ Perfect palindromes score 1.0")
print("✓ Non-palindromes score < 1.0")

## 10. Creating Custom Palindromes

Generate palindromic canons from any theme.

In [ ]:
# Create custom theme
custom_theme = stream.Part()
melody = ['G4', 'F#4', 'E4', 'D4', 'C4', 'B3', 'A3', 'G3']
rhythms = [1.0, 0.5, 0.5, 1.0, 1.5, 0.5, 0.75, 0.75]

for p, r in zip(melody, rhythms):
    custom_theme.append(note.Note(p, quarterLength=r))

# Create palindrome
custom_canon = assemble_crab_from_theme(custom_theme)

print("Custom Canon:")
print(f"Theme notes: {len(custom_theme.notes)}")
print(f"Canon parts: {len(custom_canon.parts)}")
print(f"Total duration: {custom_canon.duration.quarterLength}q")

# Verify
is_palin = is_time_palindrome(custom_canon)
score = palindrome_score(custom_canon)

print(f"\nIs palindrome: {is_palin}")
print(f"Palindrome score: {score:.3f}")

## Summary

This notebook explored:
- ✓ Time palindromes and pairwise symmetry
- ✓ Bach's Crab Canon structure
- ✓ Quantifying symmetry with statistics
- ✓ Visualizing palindromic relationships
- ✓ Temporal vs. pitch symmetry
- ✓ Palindrome quality metrics
- ✓ Creating custom palindromic canons

**Key insights:**
1. Perfect palindromes require both temporal AND pitch symmetry
2. Bach's Crab Canon achieves 100% palindrome score
3. Symmetry can be visualized and quantified
4. Any melody can be turned into a palindromic canon